In [1]:
import datetime
import logging
import json
import typing
logging.basicConfig(level=logging.INFO)

# Load primitives

In [2]:
begin = datetime.datetime.utcnow()
import d3m.primitives
(datetime.datetime.utcnow() - begin).total_seconds()

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
INFO:summa.preprocessing.cleaner:'pattern' package not found; tag filters are not available for English
INFO:matplotlib.font_manager:font search path ['/usr/local/lib/python3.6/dist-packages/matplotlib/mpl-data/fonts/ttf', '/usr/local/lib/python3.6/dist-packages/matplotlib/mpl-data/fonts/afm', '/usr/local/lib/python3.6/dist-packages/matplotlib/mpl-data/fonts/pdfcorefonts']
INFO:matplotlib.font_manager:generated new fontManager
/src/common-primitives/common_primitives/feed_forward_neural_net.py:198: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(self._layers[i].weight, gain=nn.init.calculate_gai

12.206074

In [3]:
import d3m.index
d3m.index.search()

{'d3m.primitives.bbn.time_series.AudioReader': d3m.primitives.bbn.time_series.AudioReader,
 'd3m.primitives.bbn.time_series.BBNTfidfTransformer': d3m.primitives.bbn.time_series.BBNTfidfTransformer,
 'd3m.primitives.bbn.time_series.ChannelAverager': d3m.primitives.bbn.time_series.ChannelAverager,
 'd3m.primitives.bbn.time_series.ClusterCurveFittingKMeans': d3m.primitives.bbn.time_series.ClusterCurveFittingKMeans,
 'd3m.primitives.bbn.time_series.SegmentCurveFitter': d3m.primitives.bbn.time_series.SegmentCurveFitter,
 'd3m.primitives.bbn.time_series.SequenceToBagOfTokens': d3m.primitives.bbn.time_series.SequenceToBagOfTokens,
 'd3m.primitives.bbn.time_series.SignalDither': d3m.primitives.bbn.time_series.SignalDither,
 'd3m.primitives.bbn.time_series.SignalFramer': d3m.primitives.bbn.time_series.SignalFramer,
 'd3m.primitives.bbn.time_series.SignalMFCC': d3m.primitives.bbn.time_series.SignalMFCC,
 'd3m.primitives.bbn.time_series.UniformSegmentation': d3m.primitives.bbn.time_series.Uniform

# Load data

In [4]:
with open('/d3m/data/seed_datasets_current/uu4_SPECT/TRAIN/dataset_TRAIN/datasetDoc.json') as fp:
    doc = json.load(fp)
doc

{'about': {'datasetID': 'uu4_ST_dataset_TRAIN',
  'datasetName': 'NULL',
  'license': 'CC-BY license',
  'datasetSchemaVersion': '3.0',
  'redacted': True,
  'datasetVersion': '1.0'},
 'dataResources': [{'resID': '0',
   'resPath': 'tables/learningData.csv',
   'resType': 'table',
   'resFormat': ['text/csv'],
   'isCollection': False,
   'columns': [{'colIndex': 0,
     'colName': 'd3mIndex',
     'colType': 'integer',
     'role': ['index']},
    {'colIndex': 1,
     'colName': 'OVERALL_DIAGNOSIS',
     'colType': 'categorical',
     'role': ['suggestedTarget']},
    {'colIndex': 2,
     'colName': 'F1R',
     'colType': 'integer',
     'role': ['attribute']},
    {'colIndex': 3,
     'colName': 'F1S',
     'colType': 'integer',
     'role': ['attribute']},
    {'colIndex': 4,
     'colName': 'F2R',
     'colType': 'integer',
     'role': ['attribute']},
    {'colIndex': 5,
     'colName': 'F2S',
     'colType': 'integer',
     'role': ['attribute']},
    {'colIndex': 6,
     'colNam

In [5]:
[(c['role'], c['colName']) for c in doc['dataResources'][0]['columns']]

[(['index'], 'd3mIndex'),
 (['suggestedTarget'], 'OVERALL_DIAGNOSIS'),
 (['attribute'], 'F1R'),
 (['attribute'], 'F1S'),
 (['attribute'], 'F2R'),
 (['attribute'], 'F2S'),
 (['attribute'], 'F3R'),
 (['attribute'], 'F3S'),
 (['attribute'], 'F4R'),
 (['attribute'], 'F4S'),
 (['attribute'], 'F5R'),
 (['attribute'], 'F5S'),
 (['attribute'], 'F6R'),
 (['attribute'], 'F6S'),
 (['attribute'], 'F7R'),
 (['attribute'], 'F7S'),
 (['attribute'], 'F8R'),
 (['attribute'], 'F8S'),
 (['attribute'], 'F9R'),
 (['attribute'], 'F9S'),
 (['attribute'], 'F10R'),
 (['attribute'], 'F10S'),
 (['attribute'], 'F11R'),
 (['attribute'], 'F11S'),
 (['attribute'], 'F12R'),
 (['attribute'], 'F12S'),
 (['attribute'], 'F13R'),
 (['attribute'], 'F13S'),
 (['attribute'], 'F14R'),
 (['attribute'], 'F14S'),
 (['attribute'], 'F15R'),
 (['attribute'], 'F15S'),
 (['attribute'], 'F16R'),
 (['attribute'], 'F16S'),
 (['attribute'], 'F17R'),
 (['attribute'], 'F17S'),
 (['attribute'], 'F18R'),
 (['attribute'], 'F18S'),
 (['attribu

In [6]:
from d3m.container import Dataset
from d3m.metadata.base import ALL_ELEMENTS

dataset = Dataset.load('file:///d3m/data/seed_datasets_current/uu4_SPECT/TRAIN/dataset_TRAIN/datasetDoc.json')
dataset

Dataset(id='uu4_ST_dataset_TRAIN', name='NULL', location_uris='('file:///d3m/data/seed_datasets_current/uu4_SPECT/TRAIN/dataset_TRAIN/datasetDoc.json',)')

In [7]:
list(dataset)

['0']

In [8]:
dataset['0'].head()

,d3mIndex,OVERALL_DIAGNOSIS,F1R,F1S,F2R,F2S,F3R,F3S,F4R,F4S,...,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22
0,2,1,71,62,70,64,67,64,79,65,...,1,0,0,0,0,0,0,0,0,0
1,3,1,69,71,70,78,61,63,67,65,...,0,0,0,0,0,0,0,1,1,1
2,4,1,70,66,61,66,61,58,69,69,...,1,0,1,1,0,0,0,0,0,0
3,5,1,57,69,68,75,69,74,73,71,...,1,1,0,1,0,0,0,1,0,1
4,7,1,61,60,60,62,64,72,68,67,...,1,0,0,0,0,0,0,0,0,1


In [9]:
dataset.metadata.query(['0', ALL_ELEMENTS, 3])['semantic_types']

('http://schema.org/Integer',
 'https://metadata.datadrivendiscovery.org/types/Attribute')

In [10]:
from d3m.metadata.base import ALL_ELEMENTS

In [11]:
dict(dataset.metadata.query(['0', ALL_ELEMENTS, 1]))

{'name': 'OVERALL_DIAGNOSIS',
 'structural_type': str,
 'semantic_types': ('https://metadata.datadrivendiscovery.org/types/CategoricalData',
  'https://metadata.datadrivendiscovery.org/types/SuggestedTarget')}

In [12]:
target_types = set(dataset.metadata.query(['0', ALL_ELEMENTS, 1])['semantic_types'])
target_types.add('https://metadata.datadrivendiscovery.org/types/Target')
dataset.metadata = dataset.metadata.update(
    ['0', ALL_ELEMENTS, 1],
    {
        'semantic_types': tuple(target_types)
    }
)

# Pipeline

Reproducing pipeline from [https://gitlab.com/datadrivendiscovery/metalearning/blob/master/example.yaml](https://gitlab.com/datadrivendiscovery/metalearning/blob/master/example.yaml)

<img src="...">

In [13]:
d3m.primitives.datasets.Denormalize.__bases__

(d3m.primitive_interfaces.transformer.TransformerPrimitiveBase,)

In [14]:
hyperparams_class = d3m.primitives.datasets.Denormalize.metadata.query()['primitive_code']['class_type_arguments']['Hyperparams']
hyperparams = hyperparams_class.defaults()
step0 = d3m.primitives.datasets.Denormalize(hyperparams=hyperparams)
res = step0.produce(inputs=dataset)
assert res.has_finished
step0_produce = res.value
step0_produce

Dataset(id='uu4_ST_dataset_TRAIN', name='NULL', location_uris='('file:///d3m/data/seed_datasets_current/uu4_SPECT/TRAIN/dataset_TRAIN/datasetDoc.json',)')

In [15]:
d3m.primitives.datasets.DatasetToDataFrame.__bases__

(d3m.primitive_interfaces.transformer.TransformerPrimitiveBase,)

In [16]:
hyperparams_class = d3m.primitives.datasets.DatasetToDataFrame.metadata.query()['primitive_code']['class_type_arguments']['Hyperparams']
hyperparams = hyperparams_class.defaults()
step1 = d3m.primitives.datasets.DatasetToDataFrame(hyperparams=hyperparams)
res = step1.produce(inputs=step0_produce)
assert res.has_finished
step1_produce = res.value
step1_produce.head()

,d3mIndex,OVERALL_DIAGNOSIS,F1R,F1S,F2R,F2S,F3R,F3S,F4R,F4S,...,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22
0,2,1,71,62,70,64,67,64,79,65,...,1,0,0,0,0,0,0,0,0,0
1,3,1,69,71,70,78,61,63,67,65,...,0,0,0,0,0,0,0,1,1,1
2,4,1,70,66,61,66,61,58,69,69,...,1,0,1,1,0,0,0,0,0,0
3,5,1,57,69,68,75,69,74,73,71,...,1,1,0,1,0,0,0,1,0,1
4,7,1,61,60,60,62,64,72,68,67,...,1,0,0,0,0,0,0,0,0,1


In [17]:
d3m.primitives.data.ColumnParser.__bases__

(d3m.primitive_interfaces.transformer.TransformerPrimitiveBase,)

In [18]:
hyperparams_class = d3m.primitives.data.ColumnParser.metadata.query()['primitive_code']['class_type_arguments']['Hyperparams']
hyperparams = hyperparams_class.defaults()
step2 = d3m.primitives.data.ColumnParser(hyperparams=hyperparams)
res = step2.produce(inputs=step1_produce)
assert res.has_finished
step2_produce = res.value
step2_produce.head()

,d3mIndex,OVERALL_DIAGNOSIS,F1R,F1S,F2R,F2S,F3R,F3S,F4R,F4S,...,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22
0,2,1,71,62,70,64,67,64,79,65,...,13008020892406799985,2073085640542932003,2073085640542932003,2073085640542932003,2073085640542932003,2073085640542932003,2073085640542932003,2073085640542932003,2073085640542932003,2073085640542932003
1,3,1,69,71,70,78,61,63,67,65,...,2073085640542932003,2073085640542932003,2073085640542932003,2073085640542932003,2073085640542932003,2073085640542932003,2073085640542932003,13008020892406799985,13008020892406799985,13008020892406799985
2,4,1,70,66,61,66,61,58,69,69,...,13008020892406799985,2073085640542932003,13008020892406799985,13008020892406799985,2073085640542932003,2073085640542932003,2073085640542932003,2073085640542932003,2073085640542932003,2073085640542932003
3,5,1,57,69,68,75,69,74,73,71,...,13008020892406799985,13008020892406799985,2073085640542932003,13008020892406799985,2073085640542932003,2073085640542932003,2073085640542932003,13008020892406799985,2073085640542932003,13008020892406799985
4,7,1,61,60,60,62,64,72,68,67,...,13008020892406799985,2073085640542932003,2073085640542932003,2073085640542932003,2073085640542932003,2073085640542932003,2073085640542932003,2073085640542932003,2073085640542932003,13008020892406799985


In [19]:
d3m.primitives.data.ExtractAttributes.__bases__

(d3m.primitive_interfaces.transformer.TransformerPrimitiveBase,)

In [20]:
hyperparams_class = d3m.primitives.data.ExtractAttributes.metadata.query()['primitive_code']['class_type_arguments']['Hyperparams']
hyperparams = hyperparams_class.defaults()
step3 = d3m.primitives.data.ExtractAttributes(hyperparams=hyperparams)
res = step3.produce(inputs=step2_produce)
assert res.has_finished
step3_produce = res.value
step3_produce.head()

,F1R,F1S,F2R,F2S,F3R,F3S,F4R,F4S,F5R,F5S,...,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22
0,71,62,70,64,67,64,79,65,70,69,...,13008020892406799985,2073085640542932003,2073085640542932003,2073085640542932003,2073085640542932003,2073085640542932003,2073085640542932003,2073085640542932003,2073085640542932003,2073085640542932003
1,69,71,70,78,61,63,67,65,59,59,...,2073085640542932003,2073085640542932003,2073085640542932003,2073085640542932003,2073085640542932003,2073085640542932003,2073085640542932003,13008020892406799985,13008020892406799985,13008020892406799985
2,70,66,61,66,61,58,69,69,72,68,...,13008020892406799985,2073085640542932003,13008020892406799985,13008020892406799985,2073085640542932003,2073085640542932003,2073085640542932003,2073085640542932003,2073085640542932003,2073085640542932003
3,57,69,68,75,69,74,73,71,57,61,...,13008020892406799985,13008020892406799985,2073085640542932003,13008020892406799985,2073085640542932003,2073085640542932003,2073085640542932003,13008020892406799985,2073085640542932003,13008020892406799985
4,61,60,60,62,64,72,68,67,74,68,...,13008020892406799985,2073085640542932003,2073085640542932003,2073085640542932003,2073085640542932003,2073085640542932003,2073085640542932003,2073085640542932003,2073085640542932003,13008020892406799985


In [21]:
d3m.primitives.data.CastToType.__bases__

(d3m.primitive_interfaces.transformer.TransformerPrimitiveBase,)

In [22]:
hyperparams_class = d3m.primitives.data.CastToType.metadata.query()['primitive_code']['class_type_arguments']['Hyperparams']
hyperparams = hyperparams_class.defaults()
step4 = d3m.primitives.data.CastToType(hyperparams=hyperparams)
res = step4.produce(inputs=step3_produce)
assert res.has_finished
step4_produce = res.value
step4_produce.head()

,F1R,F1S,F2R,F2S,F3R,F3S,F4R,F4S,F5R,F5S,...,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22
0,71,62,70,64,67,64,79,65,70,69,...,13008020892406799985,2073085640542932003,2073085640542932003,2073085640542932003,2073085640542932003,2073085640542932003,2073085640542932003,2073085640542932003,2073085640542932003,2073085640542932003
1,69,71,70,78,61,63,67,65,59,59,...,2073085640542932003,2073085640542932003,2073085640542932003,2073085640542932003,2073085640542932003,2073085640542932003,2073085640542932003,13008020892406799985,13008020892406799985,13008020892406799985
2,70,66,61,66,61,58,69,69,72,68,...,13008020892406799985,2073085640542932003,13008020892406799985,13008020892406799985,2073085640542932003,2073085640542932003,2073085640542932003,2073085640542932003,2073085640542932003,2073085640542932003
3,57,69,68,75,69,74,73,71,57,61,...,13008020892406799985,13008020892406799985,2073085640542932003,13008020892406799985,2073085640542932003,2073085640542932003,2073085640542932003,13008020892406799985,2073085640542932003,13008020892406799985
4,61,60,60,62,64,72,68,67,74,68,...,13008020892406799985,2073085640542932003,2073085640542932003,2073085640542932003,2073085640542932003,2073085640542932003,2073085640542932003,2073085640542932003,2073085640542932003,13008020892406799985


In [23]:
d3m.primitives.sklearn_wrap.SKImputer.__bases__

(d3m.primitive_interfaces.unsupervised_learning.UnsupervisedLearnerPrimitiveBase,)

In [24]:
hyperparams_class = d3m.primitives.sklearn_wrap.SKImputer.metadata.query()['primitive_code']['class_type_arguments']['Hyperparams']
hyperparams = hyperparams_class.defaults()
step5 = d3m.primitives.sklearn_wrap.SKImputer(hyperparams=hyperparams)
step5.set_training_data(inputs=step4_produce)
res = step5.fit()
assert res.has_finished
assert res.value is None
res = step5.produce(inputs=step4_produce)
assert res.has_finished
step5_produce = res.value

In [25]:
d3m.primitives.data.ExtractTargets.__bases__

(d3m.primitive_interfaces.transformer.TransformerPrimitiveBase,)

In [26]:
hyperparams_class = d3m.primitives.data.ExtractTargets.metadata.query()['primitive_code']['class_type_arguments']['Hyperparams']
hyperparams = hyperparams_class.defaults()
step6 = d3m.primitives.data.ExtractTargets(hyperparams=hyperparams)
res = step6.produce(inputs=step2_produce)
assert res.has_finished
step6_produce = res.value
step6_produce.head()

,OVERALL_DIAGNOSIS
0,1
1,1
2,1
3,1
4,1


In [27]:
d3m.primitives.data.CastToType.__bases__

(d3m.primitive_interfaces.transformer.TransformerPrimitiveBase,)

In [28]:
hyperparams_class = d3m.primitives.data.CastToType.metadata.query()['primitive_code']['class_type_arguments']['Hyperparams']
hyperparams = hyperparams_class.defaults()
step7 = d3m.primitives.data.CastToType(hyperparams=hyperparams)
res = step7.produce(inputs=step6_produce)
assert res.has_finished
step7_produce = res.value
step7_produce.head()

,OVERALL_DIAGNOSIS
0,1
1,1
2,1
3,1
4,1


In [29]:
d3m.primitives.sklearn_wrap.SKRandomForestClassifier.__bases__

(d3m.primitive_interfaces.supervised_learning.SupervisedLearnerPrimitiveBase,)

In [30]:
hyperparams_class = d3m.primitives.sklearn_wrap.SKRandomForestClassifier.metadata.query()['primitive_code']['class_type_arguments']['Hyperparams']
hyperparams = hyperparams_class.defaults()
step8 = d3m.primitives.sklearn_wrap.SKRandomForestClassifier(hyperparams=hyperparams)
step8.set_training_data(inputs=step5_produce, outputs=step7_produce)
res = step8.fit()
assert res.has_finished
assert res.value is None
res = step8.produce(inputs=step5_produce)
assert res.has_finished
step8_produce = res.value

/src/sklearn-wrap/sklearn_wrap/SKRandomForestClassifier.py:182: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._clf.fit(self._training_inputs, self._training_outputs)


In [33]:
step8_produce

array(['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1',
       '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1',
       '1', '1', '1', '1', '1', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '1', '1', '1', '1', '1', '1', '1', '1',
       '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1',
       '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1',
       '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1',
       '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1',
       '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1',
       '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1',
       '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1',
       '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1',
       '1', '1', '1', '1', '1', '1', '0', '0', '0', '0', '0', '0

In [31]:
d3m.primitives.data.ConstructPredictions.__bases__

(d3m.primitive_interfaces.base.PrimitiveBase,)

In [32]:
hyperparams_class = d3m.primitives.data.ConstructPredictions.metadata.query()['primitive_code']['class_type_arguments']['Hyperparams']
hyperparams = hyperparams_class.defaults()
step9 = d3m.primitives.data.ConstructPredictions(hyperparams=hyperparams)
step9.set_training_data(inputs=step2_produce)
res = step9.fit()
assert res.has_finished
assert res.value is None
res = step9.produce(inputs=step2_produce, targets=step8_produce)
assert res.has_finished
step9_produce = res.value

KeyError: 'dimension'